In [28]:
import pandas as pd

def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()

def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

### Buy the position when:

 * weekly RSI(30)  ≥  daily RSI(30)  >  70
 * Close  >  MA(10)  >  MA(20)  >  MA(50)  >  MA(100)

### Close the position when:
 * Daily close is more than 2% below MA(10)
 * 8% fixed stop loss is hit

In [29]:
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply


class System(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70
    
    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()

In [10]:
priceHist=pd.read_csv(r'C:\Users\lesap\Documents\GitHub\tensortrade\priceHistory.csv')

In [25]:
priceHist.drop('Unnamed: 0', axis=1, inplace=True)
priceHist.set_index('date', inplace=True)

KeyError: "['Unnamed: 0'] not found in axis"

In [31]:
priceHist.rename(columns={
    'open':'Open',
    'high':'High',
    'low':'Low',
    'close':'Close',
    'volume':'Volume'
}, inplace=True)

In [32]:
priceHist.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

In [34]:
priceHist.index=pd.to_datetime(priceHist.index)

In [36]:
priceHist.head()

,Open,High,Low,Close,Volume
date,,,,,
2018-05-15 06:00:00,8733.86,8796.68,8707.28,8740.99,559.93
2018-05-15 07:00:00,8740.99,8766.00,8721.11,8739.00,273.58
2018-05-15 08:00:00,8739.00,8750.27,8660.53,8728.49,917.79
2018-05-15 09:00:00,8728.49,8754.40,8701.35,8708.32,182.62
2018-05-15 10:00:00,8708.32,8865.00,8695.11,8795.90,1260.69


In [41]:
sample = priceHist[:1000]

In [42]:
from backtesting import Backtest

backtest = Backtest(priceHist, System, cash=90_000, commission=.002)
backtestSample = Backtest(sample, System, cash=90_000, commission=.002)

c:\python39\lib\site-packages\backtesting\backtesting.py:1057: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  if len(data.columns & {'Open', 'High', 'Low', 'Close', 'Volume'}) != 5:
c:\python39\lib\site-packages\backtesting\backtesting.py:1042: FutureWarning: Index.is_all_dates is deprecated, will be removed in a future version.  check index.inferred_type instead
  if (not data.index.is_all_dates and
c:\python39\lib\site-packages\backtesting\backtesting.py:1057: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  if len(data.columns & {'Open', 'High', 'Low', 'Close', 'Volume'}) != 5:


In [43]:
stats = backtest.run()
statsSample= backtestSample.run()

c:\python39\lib\site-packages\backtesting\lib.py:285: FutureWarning: Index.is_all_dates is deprecated, will be removed in a future version.  check index.inferred_type instead
  if not result.index.is_all_dates:
c:\python39\lib\site-packages\backtesting\lib.py:287: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  result = result.reindex(index=series.index | resampled.index,
c:\python39\lib\site-packages\backtesting\lib.py:285: FutureWarning: Index.is_all_dates is deprecated, will be removed in a future version.  check index.inferred_type instead
  if not result.index.is_all_dates:
c:\python39\lib\site-packages\backtesting\lib.py:287: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  result = result.reindex(index=series.index | resampled

In [44]:
stats

Start                     2018-05-15 06:00:00
End                       2021-01-08 00:00:00
Duration                    968 days 18:00:00
Exposure Time [%]                    0.172043
Equity Final [$]                 84606.088160
Equity Peak [$]                  93971.909640
Return [%]                          -5.993235
Buy & Hold Return [%]              352.537756
Return (Ann.) [%]                   -2.298760
Volatility (Ann.) [%]                3.219505
Sharpe Ratio                         0.000000
Sortino Ratio                        0.000000
Calmar Ratio                         0.000000
Max. Drawdown [%]                   -9.966618
Avg. Drawdown [%]                   -2.962097
Max. Drawdown Duration       11 days 14:00:00
Avg. Drawdown Duration        3 days 01:00:00
# Trades                                    3
Win Rate [%]                        33.333333
Best Trade [%]                       1.021913
Worst Trade [%]                     -6.261857
Avg. Trade [%]                    

In [45]:
statsSample

Start                     2018-05-15 06:00:00
End                       2018-06-25 21:00:00
Duration                     41 days 15:00:00
Exposure Time [%]                    0.000000
Equity Final [$]                 90000.000000
Equity Peak [$]                  90000.000000
Return [%]                           0.000000
Buy & Hold Return [%]              -28.394839
Return (Ann.) [%]                    0.000000
Volatility (Ann.) [%]                0.000000
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                   -0.000000
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [56]:
%%time

statsSample = backtestSample.optimize(d_rsi=range(10, 35, 5),
                  w_rsi=range(10, 35, 5),
                  level=range(30, 80, 10))
statsSample

c:\python39\lib\site-packages\backtesting\lib.py:285: FutureWarning: Index.is_all_dates is deprecated, will be removed in a future version.  check index.inferred_type instead
  if not result.index.is_all_dates:
c:\python39\lib\site-packages\backtesting\lib.py:287: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  result = result.reindex(index=series.index | resampled.index,
c:\python39\lib\site-packages\backtesting\lib.py:285: FutureWarning: Index.is_all_dates is deprecated, will be removed in a future version.  check index.inferred_type instead
  if not result.index.is_all_dates:
c:\python39\lib\site-packages\backtesting\lib.py:287: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  result = result.reindex(index=series.index | resampled

c:\python39\lib\site-packages\backtesting\lib.py:285: FutureWarning: Index.is_all_dates is deprecated, will be removed in a future version.  check index.inferred_type instead
  if not result.index.is_all_dates:
c:\python39\lib\site-packages\backtesting\lib.py:287: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  result = result.reindex(index=series.index | resampled.index,
c:\python39\lib\site-packages\backtesting\lib.py:285: FutureWarning: Index.is_all_dates is deprecated, will be removed in a future version.  check index.inferred_type instead
  if not result.index.is_all_dates:
c:\python39\lib\site-packages\backtesting\lib.py:287: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  result = result.reindex(index=series.index | resampled

c:\python39\lib\site-packages\backtesting\lib.py:285: FutureWarning: Index.is_all_dates is deprecated, will be removed in a future version.  check index.inferred_type instead
  if not result.index.is_all_dates:
c:\python39\lib\site-packages\backtesting\lib.py:287: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  result = result.reindex(index=series.index | resampled.index,
c:\python39\lib\site-packages\backtesting\lib.py:285: FutureWarning: Index.is_all_dates is deprecated, will be removed in a future version.  check index.inferred_type instead
  if not result.index.is_all_dates:
c:\python39\lib\site-packages\backtesting\lib.py:287: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  result = result.reindex(index=series.index | resampled

c:\python39\lib\site-packages\backtesting\lib.py:285: FutureWarning: Index.is_all_dates is deprecated, will be removed in a future version.  check index.inferred_type instead
  if not result.index.is_all_dates:
c:\python39\lib\site-packages\backtesting\lib.py:287: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  result = result.reindex(index=series.index | resampled.index,
c:\python39\lib\site-packages\backtesting\lib.py:285: FutureWarning: Index.is_all_dates is deprecated, will be removed in a future version.  check index.inferred_type instead
  if not result.index.is_all_dates:
c:\python39\lib\site-packages\backtesting\lib.py:287: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  result = result.reindex(index=series.index | resampled

c:\python39\lib\site-packages\backtesting\lib.py:285: FutureWarning: Index.is_all_dates is deprecated, will be removed in a future version.  check index.inferred_type instead
  if not result.index.is_all_dates:
c:\python39\lib\site-packages\backtesting\lib.py:287: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  result = result.reindex(index=series.index | resampled.index,
c:\python39\lib\site-packages\backtesting\lib.py:285: FutureWarning: Index.is_all_dates is deprecated, will be removed in a future version.  check index.inferred_type instead
  if not result.index.is_all_dates:
c:\python39\lib\site-packages\backtesting\lib.py:287: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  result = result.reindex(index=series.index | resampled

c:\python39\lib\site-packages\backtesting\lib.py:285: FutureWarning: Index.is_all_dates is deprecated, will be removed in a future version.  check index.inferred_type instead
  if not result.index.is_all_dates:
c:\python39\lib\site-packages\backtesting\lib.py:287: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  result = result.reindex(index=series.index | resampled.index,
c:\python39\lib\site-packages\backtesting\lib.py:285: FutureWarning: Index.is_all_dates is deprecated, will be removed in a future version.  check index.inferred_type instead
  if not result.index.is_all_dates:
c:\python39\lib\site-packages\backtesting\lib.py:287: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  result = result.reindex(index=series.index | resampled

c:\python39\lib\site-packages\backtesting\lib.py:285: FutureWarning: Index.is_all_dates is deprecated, will be removed in a future version.  check index.inferred_type instead
  if not result.index.is_all_dates:
c:\python39\lib\site-packages\backtesting\lib.py:287: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  result = result.reindex(index=series.index | resampled.index,
c:\python39\lib\site-packages\backtesting\lib.py:285: FutureWarning: Index.is_all_dates is deprecated, will be removed in a future version.  check index.inferred_type instead
  if not result.index.is_all_dates:
c:\python39\lib\site-packages\backtesting\lib.py:287: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  result = result.reindex(index=series.index | resampled

Wall time: 6.44 s


Start                     2018-05-15 06:00:00
End                       2018-06-25 21:00:00
Duration                     41 days 15:00:00
Exposure Time [%]                    0.000000
Equity Final [$]                 90000.000000
Equity Peak [$]                  90000.000000
Return [%]                           0.000000
Buy & Hold Return [%]              -28.394839
Return (Ann.) [%]                    0.000000
Volatility (Ann.) [%]                0.000000
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                   -0.000000
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [52]:
backtest.plot()

c:\python39\lib\site-packages\backtesting\_plotting.py:104: UserWarning: Data contains too many candlesticks to plot; downsampling to '4H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


### Result:
##### While the strategy doesn't perform as well as simple buy & hold, it does so with significantly lower exposure (time in market).
##### In conclusion, to test strategies on multiple time frames, you need to pass in OHLC data in the lowest time frame, then resample it to higher time frames, apply the indicators, then resample back to the lower time frame, filling in the in-betweens.

In [53]:
#These are the trades that would executed with this strategy.
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,3,22958,22979,26472.90012,26743.43,811.58964,0.010219,2020-12-26 21:00:00,2020-12-27 18:00:00,0 days 21:00:00
1,2,23119,23126,31613.10000,30674.72,-1876.76000,-0.029683,2021-01-02 14:00:00,2021-01-02 21:00:00,0 days 07:00:00
2,2,23137,23146,34564.36074,32399.99,-4328.74148,-0.062619,2021-01-03 08:00:00,2021-01-03 17:00:00,0 days 09:00:00


In [50]:
statsSample= backtestSample.run()

c:\python39\lib\site-packages\backtesting\lib.py:285: FutureWarning: Index.is_all_dates is deprecated, will be removed in a future version.  check index.inferred_type instead
  if not result.index.is_all_dates:
c:\python39\lib\site-packages\backtesting\lib.py:287: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  result = result.reindex(index=series.index | resampled.index,
